In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

In [ ]:
# Autload python modules by default
%load_ext autoreload
%autoreload 2

# Convert notebooks to python, so they can be loaded effiently
from utils.jupyter_loader import JupyterLoader

loader = JupyterLoader()
loader.load_all()

# Move Ordering

Before doing any further optimization, let's first check the current performance of the latest engine. For the middlegame position the `PrototypeV1Engine` engine visits `25139` nodes and needs approximately 6 seconds on our machine. 

In [ ]:
import random
import utils.min_max_tree
import IPython.display
from converted_notebooks.s09_minimax_engine import middlegame_board
from converted_notebooks.s12_simplified_evaluation import incremental_simplified_evaluator
from converted_notebooks.s14_prototype_v1 import PrototypeV1Engine

random.seed(42)

prototype_v1_engine = PrototypeV1Engine(
    evaluator=incremental_simplified_evaluator, max_look_ahead_depth=4
)

%timeit prototype_v1_engine.play(middlegame_board)

In [ ]:
tree = utils.min_max_tree.add_tree_to_engine(prototype_v1_engine)
prototype_v1_engine.play(middlegame_board)
tree.count()

One of the key reasons to do iterative deepening in the first place, is to make use of the results from previous iterations. Provided a good evaluation function is used, a reasonable assumption is that the evaluation of each move won't change much if it's analysed one depth further. This information can be used to sort moves in the next iteration. Alpha Beta pruning leads to the best results if the best move is searched first, therefore we sort the moves according to their score. 

The `_value` method will use a helper function `_get_moves` to get a sorted list of moves rather than `board.legal_moves`. The helper method takes the current board `board` and the depth `depth` as parameter and returns the list of all possible next moves, which are sorted by their value in the previous iteration. 

In [ ]:
from converted_notebooks.s11_iterative_deepening import cache_alpha_beta
from converted_notebooks.s12_simplified_evaluation import DetailedMove
from converted_notebooks.s14_prototype_v1 import PrototypeV1Engine
import chess
from chess.engine import PovScore


class MoveOrderingEngine(PrototypeV1Engine):

    @PrototypeV1Engine.depth_tracker
    @cache_alpha_beta
    def _value(
        self, board: chess.Board, depth: int, alpha: int, beta: int
    ) -> PovScore:
        if (score := self.evaluator.evaluate(board)) is not None:
            return score
        if depth == 0:
            return self._quiescence(board, alpha, beta)

        for move in self._get_moves(board, depth):
            detailedMove = DetailedMove(board, move)

            self.evaluator.push(detailedMove)
            board.push(move)
            value = self._value(board, depth - 1, -beta,
                                -alpha).pov(not board.turn)
            board.pop()
            self.evaluator.pop()

            if value >= beta:
                return PovScore(value, board.turn)
            alpha = max(alpha, value)

        return PovScore(alpha, board.turn)

    def _get_moves(self, board: chess.Board, depth: int):
        cached_moves = []
        uncached_moves = []
        for move in board.legal_moves:
            board.push(move)
            cache_key = self.cache.get_key(board)
            board.pop()

            try:
                # depth - 2 as we are looking at one move further and at the previous iteration with depth - 1
                _, value = self.cache.load_cache(cache_key, depth - 2)
                cached_moves.append((value.relative, move))
            except KeyError:
                uncached_moves.append(move)

        cached_moves.sort(reverse=False, key=lambda x: x[0])
        return [cached_move[1] for cached_move in cached_moves] + uncached_moves

If we ran the same analyse as before with the new defined engine, then the number of nodes drastically decreases from `25139` to `12911`. Unfortunately, due to the overhead of the added code, the time decreased only by approximately 1.5 seconds from `6` seconds to `4.5` seconds on our machine. Nevertheless, this is still a huge improvement.

In [ ]:
import random
import utils.min_max_tree
import IPython.display
from converted_notebooks.s09_minimax_engine import middlegame_board

random.seed(42)

move_ordering_engine = MoveOrderingEngine(
    evaluator=incremental_simplified_evaluator, max_look_ahead_depth=4
)

%timeit move_ordering_engine.play(middlegame_board)

In [ ]:
tree = utils.min_max_tree.add_tree_to_engine(move_ordering_engine)
move_ordering_engine.play(middlegame_board)
tree.count()

Sorting every move according to their score implies that the [principal variation](https://www.chessprogramming.org/Principal_Variation), therefore best line in the previous iteration, is searched first. Furthermore, at Cut Nodes the move that lead to the cut in the previous iteration, also called [refutation move](https://www.chessprogramming.org/Refutation_Move), is searched first as well. These are the two most important move criteria according to [the chessprogramming wiki](https://www.chessprogramming.org/Move_Ordering#Typical_move_ordering). The only downside of the current approach is that the best move or refutation move is not already stored in the transposition table. Changing this wouldn't improve the number of visited nodes, but the time needed to visit one node.

Next, another look should be taken on the `quiescence search`. We can calculate how many nodes are visited in the quiescence search. With `513446` the number of nodes spent in the quiescence search is by factor 40 higher than in the normal alpha beta search. This doesn't mean though that it is not worth it to optimize the `_value` function, quite the opposite is true. Every node visited less in the `_value` function will lead to a huge decrease of nodes in the `quiescence` search. Nonetheless, it might be still useful to try to optimize `_quiescence` as well.

In [ ]:
import random
import utils.min_max_tree
import IPython.display
from converted_notebooks.s09_minimax_engine import middlegame_board

random.seed(42)

move_ordering_engine = MoveOrderingEngine(
    evaluator=incremental_simplified_evaluator, max_look_ahead_depth=4
)

tree = utils.min_max_tree.add_tree_to_engine(move_ordering_engine)
move_ordering_engine.play(middlegame_board)
print(tree.count(quiesce=True))

We define a new class `QuiescenceMoveOrderingEngine` which inherits from `MoveOrderingEngine`. The constructor has one more parameter `use_static_exchange_evaluation` to determine if the more exact, but also more expensive static exchange evaluation should be used or not. The `_quiescence` method uses a helper method `_get_quiescence_detailed_moves` to get all moves that should be checked in sorted order. Optimizations like delta pruning or static exchange evaluation are moved into that function as well. 

In [ ]:
from converted_notebooks.s12_simplified_evaluation import IncrementalEvaluator
from chess.engine import Score, PovScore


class QuiescenceMoveOrderingEngine(MoveOrderingEngine):

    def __init__(
        self,
        evaluator: IncrementalEvaluator,
        max_look_ahead_depth,
        use_static_exchange_evaluation=False
    ):
        super().__init__(evaluator, max_look_ahead_depth)
        self.use_static_exchange_evaluation = use_static_exchange_evaluation

    @PrototypeV1Engine.depth_tracker
    def _quiescence(self, board: chess.Board, alpha: Score, beta: Score) -> int:
        stand_pat = self.evaluator.get_score().relative

        if stand_pat >= beta:
            return PovScore(beta, board.turn)
        alpha = max(alpha, stand_pat)

        for detailed_move in self._get_quiescence_detailed_moves(
            board, stand_pat, alpha, beta
        ):
            self.evaluator.push(detailed_move)
            board.push(detailed_move.move)
            value = self._quiescence(board, -beta, -alpha).pov(not board.turn)
            board.pop()
            self.evaluator.pop()

            if value.is_mate():
                value = self._rise_mate(value)

            if value >= beta:
                return PovScore(value, board.turn)
            alpha = max(alpha, value)
        return PovScore(alpha, board.turn)

The new `_get_quiescence_detailed_moves` method takes the board `board`, the standing pat `stand_pat` and the alpha and beta value `alpha` and `beta` as parameters and returns a sorted list of next moves. As before, only captures are considered and some of these are pruned by delta pruning or static exchange evaluation. The moves are sorted by [MVV-LVA](https://www.chessprogramming.org/MVV-LVA), which stands for most valuable victim - least valuable attacker. Therefore, the moves are first sorted by the most valuable captured piece and in case of an equal captured piece then sorted by the least valuable attacker.

In [ ]:
def _get_quiescence_detailed_moves(
    self, board: chess.Board, stand_pat: int, alpha: int, beta: int
) -> list[chess.Move]:
    moves = []
    for move in board.generate_legal_captures():
        detailedMove = DetailedMove(board, move)

        capturedPieceValue = self.evaluator.piece_values[
            detailedMove.capturedPiece.piece.piece_type]

        movedPieceValue = self.evaluator.piece_values[
            detailedMove.movedPiece.piece.piece_type]

        if move.promotion is None:
            if self._can_delta_prune(stand_pat, alpha, capturedPieceValue):
                continue

            if self._bad_capture(
                board, detailedMove, capturedPieceValue, movedPieceValue
            ):
                continue

        moves.append((capturedPieceValue, movedPieceValue, detailedMove))

    # Oder by MVV-LVA (most valuable victim first, least valuable attacker second)
    moves.sort(reverse=True, key=lambda x: (x[0], -x[1]))

    return [move[2] for move in moves]


QuiescenceMoveOrderingEngine._get_quiescence_detailed_moves = _get_quiescence_detailed_moves

The method `_can_delta_prune` has been rewritten to use the value of the captured piece that was already calculated in `_get_quiescence_detailed_moves` by taking it as a parameter.

In [ ]:
from chess.engine import Cp


def _can_delta_prune(
    self, stand_pat: Score, alpha: Score, captured_piece_value: int
):
    POTENTIAL_POSITION_ADVANTAGE = 200
    best_alpha = Cp(
        stand_pat.score() + captured_piece_value + POTENTIAL_POSITION_ADVANTAGE
    )
    return best_alpha < alpha


QuiescenceMoveOrderingEngine._can_delta_prune = _can_delta_prune

Instead of directly calling `_see_capture` in `_get_quiescence_detailed_moves` another method `_bad_capture` is called first, which returns a boolean indicating if the move should be skipped. It takes a board `board`, the detailed move `detailed_move` and the information about the value of the captured and moved piece as parameters `captured_piece_value` and `moved_piece_value`. It first checks whether we could lose any material. If this is not the case, the move is not classified as bad. If static exchange evaluation is disabled, we consider moves where we could lose material only as not bad if the captured piece is actually not defended at all. If static exchange evaluation is enabled we use this to check the left captures the same way as before. The method `_see_capture` was rewritten as well to accept `captured_piece_value` and `moved_piece_value` as parameters.

In [ ]:
def _bad_capture(
    self,
    board: chess.Board,
    detailed_move: DetailedMove,
    captured_piece_value: int,
    moved_piece_value: int
):
    value = captured_piece_value - moved_piece_value
    if value >= 0:
        return False

    if self.use_static_exchange_evaluation:
        return self._see_capture(
            board, detailed_move, captured_piece_value, moved_piece_value
        ) < 0
    return board.is_attacked_by(
        not board.turn, detailed_move.placedPiece.square
    )


QuiescenceMoveOrderingEngine._bad_capture = _bad_capture


def _see_capture(
    self,
    board: chess.Board,
    detailed_move: DetailedMove,
    captured_piece_value: int,
    moved_piece_value: int
) -> int:
    board.push(detailed_move.move)
    value = captured_piece_value - self._see(
        board, detailed_move.placedPiece.square
    )
    board.pop()
    return value


QuiescenceMoveOrderingEngine._see_capture = _see_capture

A test of the engine with static exchange evaluation enabled shows that the number of visited nodes is decreased from `513446` to `468023`, so approximately 10% of the nodes are saved. The time slightly decreased by a quarter second to `4.25` seconds.

In [ ]:
import random
from converted_notebooks.s09_minimax_engine import middlegame_board
from converted_notebooks.s12_simplified_evaluation import incremental_simplified_evaluator

random.seed(42)

quiescence_move_ordering_engine = QuiescenceMoveOrderingEngine(
    evaluator=incremental_simplified_evaluator,
    max_look_ahead_depth=4,
    use_static_exchange_evaluation=True
)

%timeit quiescence_move_ordering_engine.play(middlegame_board)

In [ ]:
import utils.min_max_tree

tree = utils.min_max_tree.add_tree_to_engine(quiescence_move_ordering_engine)
quiescence_move_ordering_engine.play(middlegame_board)
print(tree.count(quiesce=True))

With static exchange evaluation disabled the number of visited nodes decreases to `319376`, because more nodes have been pruned by the simpler metric. The time decreases drastically to `2.85` seconds. This is not only because of the fewer nodes, but also because the simple metric is faster to calculate. The downside is that potential good moves are not evaluated anymore.

In [ ]:
import random
from converted_notebooks.s09_minimax_engine import middlegame_board
from converted_notebooks.s12_simplified_evaluation import incremental_simplified_evaluator

random.seed(42)

quiescence_move_ordering_engine = QuiescenceMoveOrderingEngine(
    evaluator=incremental_simplified_evaluator,
    max_look_ahead_depth=4,
    use_static_exchange_evaluation=False
)

%timeit quiescence_move_ordering_engine.play(middlegame_board)

In [ ]:
import utils.min_max_tree

tree = utils.min_max_tree.add_tree_to_engine(quiescence_move_ordering_engine)
quiescence_move_ordering_engine.play(middlegame_board)
print(tree.count(quiesce=True))

The static exchange evaluation can be improved further though regarding its speed. Currently, it uses the expensive `push` and `pop` calls of the board. It is possible to only remove the capturing piece. For this to work, the board needs to be copied first, so no piece is permanently removed be the method. Then it's necessary to keep track of the captured piece and side to turn to correctly calculate the static evaluation score. One downside of this optimized function is that it ignores some chess rules in regard of performance. For instance, it is not checked whether a piece might be pinned and therefore cannot capture. But as static exchange evaluation is only an estimation, this is a trade off that can be done.

In [ ]:
class OptimizedQuiescenceMoveOrderingEngine(QuiescenceMoveOrderingEngine):

    def _see_capture(
        self,
        original_board: chess.Board,
        detailed_move: DetailedMove,
        captured_piece_value: int,
        moved_piece_value: int
    ) -> int:
        board = original_board.copy(stack=False)

        board.remove_piece_at(detailed_move.movedPiece.square)
        value = captured_piece_value - self._see(
            board,
            detailed_move.placedPiece.square,
            moved_piece_value,
            not board.turn
        )
        return value

    def _see(
        self,
        board: chess.Board,
        square: chess.Square,
        attacked_piece_value: int,
        turn: chess.Color
    ) -> int:
        attackers = [
            (square, self.evaluator.piece_values[board.piece_type_at(square)])
            for square in board.attackers(turn, square)
        ]
        if not attackers:
            return 0

        attacker_square, attacker_piece_value = min(attackers, key=lambda x: x[1])

        board.remove_piece_at(attacker_square)
        value = max(
            0,
            attacked_piece_value -
            self._see(board, square, attacker_piece_value, not turn)
        )
        return value

A test shows that the time decreased from about `4.25` seconds to `3.45` seconds on our machine.

In [ ]:
import random
from converted_notebooks.s09_minimax_engine import middlegame_board
from converted_notebooks.s12_simplified_evaluation import incremental_simplified_evaluator

random.seed(42)

optimized_quiescence_move_ordering_engine = OptimizedQuiescenceMoveOrderingEngine(
    evaluator=incremental_simplified_evaluator,
    max_look_ahead_depth=4,
    use_static_exchange_evaluation=True
)

%timeit optimized_quiescence_move_ordering_engine.play(middlegame_board)

# Principal Variation Search

The last enhancement we implement is a new search algorithm. The idea behind the [principle variation search (PVS)](https://www.chessprogramming.org/Principal_Variation_Search) is that we use move ordering and thus the first move we analyse is most likely already the best one. Therefore, we don't fully analyse the other moves, but check first with a quicker test if our assumption is true. We use a [null window](https://www.chessprogramming.org/Null_Window) for this. This means we set `alpha` to `beta - 1`. The consequence of this is first that there will be more cutoffs and the search is generally faster. But we never get an exact score from such a search, we only know if the current move is better or worse than our current best move. If the current move is actually better, another search with the normal window is necessary to find the exact score. So PVS can actually be slower because of the test and the following research if we don't pick a good move as the first move.

We start by defining a new class.

In [ ]:
class PrincipalVariationSearch(QuiescenceMoveOrderingEngine):
    pass

Next a method `_zero_width_search` is defined to perform the zero width search. It has the same signature as `_value` except that it only has `beta` and no `alpha` value. Alpha is derived from beta with `alpha = beta - 1`. The rest of the implementation is very similar to `_value` except that we don't need to keep track of the current best score. If any score would increase `alpha` it would as a consequence directly cause a beta cutoff.

In [ ]:
@PrototypeV1Engine.depth_tracker
def _zero_width_search(
    self, board: chess.Board, depth: int, beta: Score
) -> PovScore:
    if beta.is_mate():
        alpha = beta.increased_mate(1)
    else:
        alpha = Cp(beta.score() - 1)

    if (score := self.evaluator.evaluate(board)) is not None:
        return self._adjust_mate(score)
    if depth == 0:
        return self._quiescence(board, alpha, beta)

    for move in board.generate_legal_moves():
        detailedMove = DetailedMove(board, move)

        self.evaluator.push(detailedMove)
        board.push(move)
        value = self._zero_width_search(board, depth - 1,
                                        -alpha).pov(not board.turn)
        board.pop()
        self.evaluator.pop()

        if value >= beta:
            return PovScore(value, board.turn)
    return PovScore(alpha, board.turn)


PrincipalVariationSearch._zero_width_search = _zero_width_search

The `_value` does a full search for every move until one move raises alpha. For the other moves a zero width search is done. If any move would actually raise alpha, a full research needs to be done.

In [ ]:
from converted_notebooks.s11_iterative_deepening import cache_alpha_beta
from converted_notebooks.s12_simplified_evaluation import DetailedMove
from converted_notebooks.s14_prototype_v1 import PrototypeV1Engine


@PrototypeV1Engine.depth_tracker
@cache_alpha_beta
def _value(
    self, board: chess.Board, depth: int, alpha: Score, beta: Score
) -> PovScore:
    if (score := self.evaluator.evaluate(board)) is not None:
        return self._adjust_mate(score)
    if depth == 0:
        return self._quiescence(board, alpha, beta)

    search_principal_variation = True
    for move in self._get_moves(board, depth):
        detailedMove = DetailedMove(board, move)

        self.evaluator.push(detailedMove)
        board.push(move)

        if search_principal_variation:
            value = self._value(board, depth - 1, -beta,
                                -alpha).pov(not board.turn)
        else:
            value = self._zero_width_search(board, depth - 1,
                                            -alpha).pov(not board.turn)
            if alpha < value < beta:
                value = self._value(board, depth - 1, -beta,
                                    -alpha).pov(not board.turn)

        board.pop()
        self.evaluator.pop()

        if value >= beta:
            return PovScore(value, board.turn)

        if value > alpha:
            alpha = value
            search_principal_variation = False

    return PovScore(alpha, board.turn)


PrincipalVariationSearch._value = _value

The engine with static exchange evaluation disabled now only needs `1.75` seconds instead of `2.85` seconds.

In [ ]:
import random
import utils.min_max_tree
import IPython.display
from converted_notebooks.s09_minimax_engine import middlegame_board
from converted_notebooks.s12_simplified_evaluation import incremental_simplified_evaluator

random.seed(42)

principal_variation_search_engine = PrincipalVariationSearch(
    evaluator=incremental_simplified_evaluator,
    max_look_ahead_depth=4,
    use_static_exchange_evaluation=False
)

%timeit principal_variation_search_engine.play(middlegame_board)

With static exchange evaluation enabled the engine needs `2.8` seconds instead of `3.45` seconds.

In [ ]:
principal_variation_search_engine = PrincipalVariationSearch(
    evaluator=incremental_simplified_evaluator,
    max_look_ahead_depth=4,
    use_static_exchange_evaluation=True
)

%timeit principal_variation_search_engine.play(middlegame_board)